# Agregagador de CSV's

Agrega os csv's de cada órgão em um só csv (remuneracao_servidores.csv), criando os atributos 'ORGAO', 'ANO' e 'MES'. 

### Imports

In [1]:
import pandas as pd
import os
import re
import gc
import csv
import sys

csv.field_size_limit(sys.maxsize)

131072

## Funções

### Lista Diretórios

In [2]:
def lista_dirs(path):
    dirs=[]
    # r = raiz, d = diretorios, f = arquivos
    for r, d, f in os.walk(path):
        for direc in d:
            if '.txt' not in direc:
                dirs.append(os.path.join(r, direc))
    return dirs

### Lista os arquivos de um diretório

In [3]:
def lista_arqs(path):
    files = []
    # r = raiz, d = diretorios, f = arquivos
    for r, d, f in os.walk(path):
        for file in f:
            if '.csv' in file:
                f_out = os.path.join(r, file)
                files.append(f_out)
    return files

### Cria o dataframe de cada órgão

In [4]:
def cria_df_orgao(path):
    skip_13 = True
    
    files = lista_arqs(path)
    df_orgao = pd.DataFrame()
    for f in files:
        if '.csv' in f:
            if f.endswith("13.csv"):
                continue
            df_mes = pd.read_csv(f, sep=';', header=0, decimal=',', engine='python',error_bad_lines=False)
            
            file_name = f.split("_")[1]
            orgao_name = path.split("/")[2]
            
            mes = file_name[4:6]
            ano = file_name[0:4]
            
            if skip_13 and mes == '13':
                continue
            else:
                data_string = ano+'-'+mes+'-01'
                df_mes["DATA"] = pd.to_datetime(data_string)

                df_orgao = pd.concat([df_orgao, df_mes], ignore_index=True)
                df_orgao["ORGAO"] = orgao_name
    try:
        df_orgao.drop(['Unnamed: 10'], axis=1, inplace=True)
    except:
        print('Sem Coluna Unnamed')
    return df_orgao

### Cria o dataframe geral

In [5]:
def cria_df_geral(path):
    df_geral = pd.DataFrame()
    dirs = lista_dirs(path)
    for d in dirs:
        if (str(d) == "../csv/PRODAM"):
            continue
        print("Processando diretório:", str(d))
        nome_dir = d.split("/")[1]
        df_orgao = cria_df_orgao(d)
        df_geral = pd.concat([df_geral,df_orgao], ignore_index=True)
        df_geral = trata_nan(df_geral)
    return df_geral

### Trata os valores NaN

In [6]:
def trata_nan(df):
    for col in df.columns:
        if(df[col].dtype == 'object'):
            df[col] = df[col].fillna('-')
        if(df[col].dtype == 'float64'):
            df[col] = df[col].fillna(0.00)
    
    return df

### Salva o dataframe

In [ ]:
def salva_df(df):
    path = r'../ds/'
    if (not os.path.exists(path)):
        os.mkdir(path)
    df.to_csv(path + r'remuneracao_servidores.csv')

## Main

In [ ]:
path = '../csv/'
gc.enable()
df_geral = cria_df_geral(path)
salva_df(df_geral)
df_geral

Processando diretório: ../csv/FEH
Processando diretório: ../csv/SETRAB
Processando diretório: ../csv/CASA MILITAR
Processando diretório: ../csv/JUCEA
Processando diretório: ../csv/SEJEL
Processando diretório: ../csv/SEAD-PENSÃO HANSENIANOS
Processando diretório: ../csv/CGE
Processando diretório: ../csv/FUNDAÇÃO VILA OLIMPICA
Sem Coluna Unnamed
Processando diretório: ../csv/FUNDAÇÃO AMAZONPREV
Processando diretório: ../csv/PM-ATIVOS


Skipping line 4328: unexpected end of data
Skipping line 4264: unexpected end of data
Skipping line 4343: unexpected end of data
Skipping line 4265: unexpected end of data
Skipping line 4307: unexpected end of data
Skipping line 4266: unexpected end of data
Skipping line 4270: unexpected end of data
Skipping line 4261: unexpected end of data
Skipping line 4333: unexpected end of data


Processando diretório: ../csv/ERGSP
Processando diretório: ../csv/SNPH
Processando diretório: ../csv/IMPRENSA OFICIAL
Processando diretório: ../csv/FAPEAM
Processando diretório: ../csv/IDAM
Processando diretório: ../csv/SEIND
Sem Coluna Unnamed
Processando diretório: ../csv/AMAZONPREV
Processando diretório: ../csv/SUSAM
Processando diretório: ../csv/OUVIDORIA GERAL
Processando diretório: ../csv/SEDUC
Processando diretório: ../csv/POLICIA CIVIL
Processando diretório: ../csv/FEI
Processando diretório: ../csv/CB-CIVIS
Processando diretório: ../csv/SERGB
Sem Coluna Unnamed
Processando diretório: ../csv/SSP
Processando diretório: ../csv/SEFAZ
Processando diretório: ../csv/FCECON
Processando diretório: ../csv/ADS
Processando diretório: ../csv/CASA CIVIL DO GOVERNO
Processando diretório: ../csv/SEPED
Processando diretório: ../csv/FHAJ
Processando diretório: ../csv/SEAD-PENSÃO ESPECIAL II
Processando diretório: ../csv/SEMA
Processando diretório: ../csv/IPAAM
Processando diretório: ../csv/SPF
P